# Workflow: processing, searching and retrieving observations

This tutorial demonstrates how OpenGHG can be used to process new measurement data, search the data present and to retrieve this for analysis and visualisation.

### Check installation

This tutorial assumes that you have installed `openghg`. To ensure install has been successful you can open an `ipython` console and try to import this module.

In a terminal type:

```bash
$ ipython
```

Then import `openghg` and print the version string associated with the version you have installed. If you get something like the below `openghg` is installed correctly.

```ipython
In [1]: import openghg
In [2]: openghg.__version__
Out[2]: '0.0.1'
```

If you get an ``ImportError`` please go back to the [install section of the documentation](https://docs.openghg.org/install.html).

### Jupyter notebooks

If you haven't used Jupyter notebooks before please see [this introduction](https://realpython.com/jupyter-notebook-introduction/).

## 1. Setting up an object store

The OpenGHG platform uses what's called an *object store* to save data. Any saved data has been processed into a standardised format, assigned universally unique identifiers (UUIDs) and stored alongside associated metadata (such as site and species details). Storing data in this way allows for fast retrieval and efficient searching.

When using OpenGHG on a local machine the location of the object store is set using an `OPENGHG_PATH` environment variable (explained below) and this can be any directory on your local system.

For this tutorial, we will create a temporary object store which we can add new data to. For the purposes of this tutorial this path is fine but as it is a temporary directory it may not survive a reboot of the computer. 

The `OPENGHG_PATH` environment variable can be set up in the following way.

In [1]:
import os
import tempfile

tmp_dir = tempfile.TemporaryDirectory()
os.environ["OPENGHG_PATH"] = tmp_dir.name   # temporary directory

When creating your own longer term object store we recommend a path such as ``~/openghg_store`` which will create the object store in your home directory in a directory called ``openghg_store``. If you want this to be a permanent location this can be added to your "~/.bashrc" or "~/.bash_profile" file depending on the system being used. e.g. as

```bash
 export OPENGHG_PATH="$HOME/openghg_store"
```

## 2. Adding and standardising data

### Data types

Within OpenGHG there are several data types which can be processed and stored within the object store. This includes data from the AGAGE, DECC, NOAA, LondonGHG, BEAC2ON networks.

When uploading a new data file, the data type must be specified alongside some additional details so OpenGHG can recognise the format and the correct standardisation can occur. The details needed will vary by the type of data being uploaded but will often include the measurement reference (e.g. a site code) and the name of any network.

For the full list of accepted data types and necessary details see *++here (Add documentation page when created)++ (or generate the list using this function..)*.

### DECC network

We will start by adding data to the object store from a surface site within the DECC network. Here we have accessed a subset of data from the Bilsdale site (site code "BSD") in the UK.

In [2]:
bsd_filepaths = ["../data/DECC/bsd.picarro.1minute.42m.min.dat", "../data/DECC/bsd.picarro.1minute.108m.min.dat", "../data/DECC/bsd.picarro.1minute.248m.min.dat"]


As this data is measured in-situ, this is classed as a surface site and we need to be the `ObsSurface` class to interpret this data. We can pass our list of files to the `read_file` method associated within the `ObsSurface` class function, also providing details on:
 - site code - `"BSD"` for Billsdale
 - type of data we want to process, known as the data type - `"CRDS"`
 - network - `"DECC"`
This is shown below:

In [3]:
from openghg.store import ObsSurface

decc_results = ObsSurface.read_file(filepath=bsd_filepaths, data_type="CRDS", site="bsd", network="DECC", overwrite=True)

/user/home/rt17603/work/environments/openghg_env/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


Processing: bsd.picarro.1minute.248m.min.dat: 100%|██████████| 3/3 [00:01<00:00,  2.70it/s]


In [4]:
print(decc_results)

defaultdict(<class 'dict'>, {'processed': {'bsd.picarro.1minute.42m.min.dat': {'ch4': 'dfff9891-96a0-4a51-99b2-f58a412e61f7', 'co2': '3853bbd9-4650-4755-bf22-80ae8dc121e8', 'co': '08ecef40-b8a4-47d3-a6d7-3aa5a34be876'}, 'bsd.picarro.1minute.108m.min.dat': {'ch4': '1c054a4b-6032-489a-a5ac-111f8074a1c0', 'co2': 'fe6882b2-e982-4998-8a3a-7614149f238e', 'co': '45f21ea7-f371-4120-8488-401ab0169756'}, 'bsd.picarro.1minute.248m.min.dat': {'ch4': '40cf5fc8-54d7-4b7b-97f9-1755de19b838', 'co2': 'cb549b52-6101-44b3-9e9a-6240e7401835', 'co': 'e32b31bd-c032-4d24-bcc8-531798fe8b76'}}})


Here this extracts the data (and metadata) from the supplied files, standardises them and adds these to our created object store.

The returned `decc_results` will give us a dictionary of how the data has been stored. The data itself may have been split into different entries, each one stored with a unique ID. Each entry is known as a *Datasource* (see below for a note on Datasources). The `decc_results` output includes details of the processed data and tells us that the data has been stored correctly.

### AGAGE data

Another data type which can be added is data from the AGAGE network. The functions that process the AGAGE data expect data to have an accompanying precisions file. For each data file we create a tuple with the data filename and the precisions filename. *Note: A simpler method of uploading these file types is planned.*

We must create a `tuple` associated with each data file to link this to a precision file:

```python
list_of_tuples = [(data1_filepath, precision1_filepath), (data2_filepath, precision2_filepath), ...]
```

The data being uploaded here is from the Cape Grim station in Australia, site code "CGO".

In [5]:
agage_tuples = [('../data/AGAGE/capegrim-medusa.18.C', '../data/AGAGE/capegrim-medusa.18.precisions.C')]

We can add these files to the object store in the same way as the DECC data but including the right keywords:
 - site code - `"CGO"` for Cape Grim
 - data type - `"GCWERKS"`
 - network - `"AGAGE"`

In [6]:
from openghg.store import ObsSurface

agage_results = ObsSurface.read_file(filepath=agage_tuples, data_type="GCWERKS", site="CGO", network="AGAGE", overwrite=True)

Processing: capegrim-medusa.18.C:   0%|          | 0/1 [00:00<?, ?it/s]

/user/home/rt17603/work/environments/openghg_env/lib/python3.8/site-packages/openghg/standardise/surface/_gcwerks.py:288: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[gas_name + " integration_flag"] = (data[column].str[1] != "-").astype(int)
/user/home/rt17603/work/environments/openghg_env/lib/python3.8/site-packages/openghg/standardise/surface/_gcwerks.py:287: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[gas_name + " status_flag"] = (data[column].str[0] != "-").astype(int)


Processing: capegrim-medusa.18.C: 100%|██████████| 1/1 [00:01<00:00,  1.65s/it]


When viewing `agage_results` there will be a large number of Datasource UUIDs shown due to the large number of gases in each data file

In [7]:
agage_results

defaultdict(dict,
            {'processed': {'capegrim-medusa.18.C': {'nf3_70m': 'c0540085-9bac-4b00-bade-80e84b2bacba',
               'cf4_70m': '783bd4d6-3452-4ce3-8c4a-d5f6a7c99747',
               'c2f6_70m': 'b6bc203f-b098-4751-9f4b-ba2e712a8c2f',
               'c3f8_70m': 'c65701cc-a6cd-40c9-b554-935c23216334',
               'c4f8_70m': '0cc00ae7-24ed-479e-8152-007effed2f57',
               'c4f10_70m': '3ec31bad-af03-43c5-967e-888b7fd0f2fc',
               'c6f14_70m': 'fc835ea3-d762-445b-a00e-e5074b03728f',
               'sf6_70m': '7c93982f-856b-4b4f-94fe-f8e710c52b64',
               'so2f2_70m': 'f0626856-0c51-4c8b-8210-eda7dca2f54c',
               'sf5cf3_70m': '30f8d6dd-3a8a-47c0-8424-d8c209f88f25',
               'hfc23_70m': '9024f0ed-e246-43e8-ae1d-3493522f42c8',
               'hfc32_70m': 'a8bae867-c95e-4c48-a20d-771274a41e3a',
               'hfc125_70m': '9111deb2-38a5-459b-8f3a-adc72cae4622',
               'hfc134a_70m': 'c8d2bf39-d1ce-4980-a9ce-cbac514bc04d'

#### A note on Datasources

Datasources are objects that are stored in the object store (++add link to object store notes++) that hold the data and metadata associated with each measurement we upload to the platform.

For example, if we upload a file that contains readings for three gas species from a single site at a specific inlet height OpenGHG    will assign this data to three different Datasources, one for each species. Metadata such as the site, inlet height, species, network etc are stored alongside the measurements for easy searching. 

Datasources can also handle multiple versions of data from a single site, so if scales or other factors change multiple versions may be stored for easy future comparison.

## 3. Searching for data

### Visualising the object store

Now that we have added data to our created object store, we can view the objects within it in a simple force graph model. To do this we use the `view_store` function from the `objectstore` submodule. Note that the cell may take a few moments to load as the force graph is created.

In the force graph the central blue node is the `ObsSurface` node. Associated with this node are all the data processed by it. The next node in the topology are networks, shown in green. In the graph you will see `DECC` and `AGAGE` nodes from the data files we have added. From these you'll see site nodes in red and then individual datasources in orange.

*Note: The object store visualisation created by this function is commented out here and won't be visible in the documentation but can be uncommented and run when you use the notebook version.*


In [8]:
from openghg.objectstore import visualise_store

visualise_store()

Now we know we have this data in the object store we can search it and retrieve data from it.

### Searching the object store

We can also search the object store by property using the `search(...)` function.

For example we can find all sites which have measurements for carbon tetrafluoride ("cf4") using the `species` keyword:

In [9]:
from openghg.retrieve import search
search(species="cf4")

Site: CGO
---------
cf4 at 70m


We could also look for details of all the data measured at the Billsdale ("BSD") site using the `site` keyword:

In [10]:
search(site="bsd")

Site: BSD
---------
ch4 at 42m, 108m, 248m
co2 at 42m, 108m, 248m
co at 42m, 108m, 248m


For this site you can see this contains details of each of the species as well as the inlet heights these were measured at.

## 4. Retrieving data 

To retrieve the standardised data from the object store there are several functions we can use which depend on the type of data we want to access.

To access the surface data we have added so far we can use the `get_obs_surface` function and pass keywords for the site code, species and inlet height to retrieve our data.

In this case we want to extract the carbon monoxide ("co") data from the Bilsdale data ("BSD") site measured at the "248m" inlet:

In [11]:
from openghg.retrieve import get_obs_surface

obs_data = get_obs_surface(site="bsd", species="co", inlet="248m")

If we view our returned `obs_data` variable this will contain:

 - `data` - The standardised data (accessed using e.g. `obs_data.data`). This is returned as an [xarray Dataset](https://xarray.pydata.org/en/stable/generated/xarray.Dataset.html).
 - `metadata` - The associated metadata (accessed using e.g. `obs_data.metadata`).

In [12]:
obs_data

ObsData(data=<xarray.Dataset>
Dimensions:                    (time: 142)
Coordinates:
  * time                       (time) datetime64[ns] 2014-01-30T11:12:30 ... ...
Data variables:
    mf                         (time) float64 202.4 203.2 205.1 ... 114.5 114.2
    mf_variability             (time) float64 5.265 6.307 8.518 ... 7.339 5.405
    mf_number_of_observations  (time) float64 26.0 26.0 25.0 ... 23.0 24.0 23.0
Attributes: (12/23)
    data_owner:           Simon O'Doherty
    data_owner_email:     s.odoherty@bristol.ac.uk
    inlet_height_magl:    248m
    comment:              Cavity ring-down measurements. Output from GCWerks
    long_name:            bilsdale
    Conditions of use:    Ensure that you contact the data owner at the outse...
    ...                   ...
    sampling_period:      60
    inlet:                248m
    port:                 9
    type:                 air
    network:              decc
    scale:                WMO-X2014A, metadata={'site': 'bsd'

First we tell `matplotlib` that we are plotting inside a Jupyter notebook, this ensures a plot with controls is created.

In [13]:
%matplotlib notebook

In [14]:
example_data = obs_data.data
mol_frac = example_data.mf
mol_frac.plot()

<IPython.core.display.Javascript object>

## 5. Ranking data

The dates that the data from Heathfield retrieved above overlap. If we want to easily retrieve the highest quality data from Heathfield over a range of dates we don't want to have to repeatedly check which was the correct inlet/instrument for a given daterange. This problem is solved using ranking. 

A given inlet on a specific instrument at a site can be given a rank for a daterange. To do this we use the `RankSources` class from the `localclient` submodule.

## Get ranking data

In [15]:
from openghg.localclient import RankSources

r = RankSources()
r.get_sources(site="bsd", species="co")

{'co_42m_picarro': {'rank_data': 'NA',
  'data_range': '2014-01-30T11:12:30_2020-12-01T22:31:30'},
 'co_108m_picarro': {'rank_data': 'NA',
  'data_range': '2014-01-30T11:12:30_2020-12-01T22:31:30'},
 'co_248m_picarro': {'rank_data': 'NA',
  'data_range': '2014-01-30T11:12:30_2020-12-01T22:31:30'}}

## Set ranking data

This tells us that there is no rank set for CO data at Bisldale. Say we want to prioritise data from different inlet heights between certain dates we can set the ranks like so

In [16]:
r.set_rank(key="co_248m_picarro", rank=1, start_date="2016-01-01", end_date="2018-01-01")
r.set_rank(key="co_42m_picarro", rank=1, start_date="2018-01-02", end_date="2019-05-30")
r.set_rank(key="co_108m_picarro", rank=1, start_date="2019-05-30", end_date="2021-11-30")

Now we can check everything was set correctlying using `get_sources` again as above

In [17]:
r.get_sources(site="bsd", species="co")

{'co_42m_picarro': {'rank_data': {'2018-01-02-00:00:00+00:00_2019-05-30-00:00:00+00:00': 1},
  'data_range': '2014-01-30T11:12:30_2020-12-01T22:31:30'},
 'co_108m_picarro': {'rank_data': {'2019-05-30-00:00:00+00:00_2021-11-30-00:00:00+00:00': 1},
  'data_range': '2014-01-30T11:12:30_2020-12-01T22:31:30'},
 'co_248m_picarro': {'rank_data': {'2016-01-01-00:00:00+00:00_2018-01-01-00:00:00+00:00': 1},
  'data_range': '2014-01-30T11:12:30_2020-12-01T22:31:30'}}

## Retrieve ranked data

We can now retrive CO data from Bilsdale and we'll get the data from the highest ranked inlets.

In [18]:
co_data = get_obs_surface(site="bsd", species="co")

This retrieves the data and creates an `ObsData` dataclass. This holds the data in an xarray `Dataset`, the metadata associated with the site and each inlet and the ranking metadata. 

Let's have a look at the rank metadata

In [19]:
co_data.metadata["rank_metadata"]

{'2018-01-02-00:00:00+00:00_2019-05-30-00:00:00+00:00': '42m',
 '2019-05-30-00:00:00+00:00_2021-11-30-00:00:00+00:00': '108m',
 '2016-01-01-00:00:00+00:00_2018-01-01-00:00:00+00:00': '248m'}

In [20]:
co_data.data

<xarray.Dataset>
Dimensions:                    (time: 162)
Coordinates:
  * time                       (time) datetime64[ns] 2014-01-30T11:12:30 ... ...
Data variables:
    mf                         (time) float64 202.4 203.2 205.1 ... 114.5 114.2
    mf_variability             (time) float64 5.265 6.307 8.518 ... 7.339 5.405
    mf_number_of_observations  (time) float64 26.0 26.0 25.0 ... 23.0 24.0 23.0
Attributes: (12/23)
    data_owner:           Simon O'Doherty
    data_owner_email:     s.odoherty@bristol.ac.uk
    inlet_height_magl:    248m
    comment:              Cavity ring-down measurements. Output from GCWerks
    long_name:            bilsdale
    Conditions of use:    Ensure that you contact the data owner at the outse...
    ...                   ...
    sampling_period:      60
    inlet:                248m
    port:                 9
    type:                 air
    network:              decc
    scale:                WMO-X2014A

In [21]:
co_data.metadata

{'42m': {'site': 'bsd',
  'instrument': 'picarro',
  'sampling_period': '60',
  'inlet': '42m',
  'port': '9',
  'type': 'air',
  'network': 'decc',
  'species': 'co',
  'scale': 'wmo-x2014a',
  'long_name': 'bilsdale',
  'data_type': 'timeseries'},
 '108m': {'site': 'bsd',
  'instrument': 'picarro',
  'sampling_period': '60',
  'inlet': '108m',
  'port': '9',
  'type': 'air',
  'network': 'decc',
  'species': 'co',
  'scale': 'wmo-x2014a',
  'long_name': 'bilsdale',
  'data_type': 'timeseries'},
 '248m': {'site': 'bsd',
  'instrument': 'picarro',
  'sampling_period': '60',
  'inlet': '248m',
  'port': '9',
  'type': 'air',
  'network': 'decc',
  'species': 'co',
  'scale': 'wmo-x2014a',
  'long_name': 'bilsdale',
  'data_type': 'timeseries'},
 'rank_metadata': {'2018-01-02-00:00:00+00:00_2019-05-30-00:00:00+00:00': '42m',
  '2019-05-30-00:00:00+00:00_2021-11-30-00:00:00+00:00': '108m',
  '2016-01-01-00:00:00+00:00_2018-01-01-00:00:00+00:00': '248m'},
 'data_owner': "Simon O'Doherty",


Now we have the highest ranked data for Bilsdale.

## 7. Cleanup

If you used the `tmp_dir` as a location for your object store at the start of the tutorial you can run the cell below to remove any files that were created.

In [22]:
tmp_dir.cleanup()

## 8. What's next?

Further tutorials will be added soon. If you want to explore the internal workings of OpenGHG please checkout the Developer API documentation, if you would like contribute to the project we welcome pull requests to both the code and the documentation. For help and guidance on contributing check our contributing page.